Doing transformation for all tables

In [0]:
from pyspark.sql.functions import from_utc_timestamp, date_format
from pyspark.sql.types import TimestampType

table_name = []
for i in dbutils.fs.ls('/mnt/bronze/SalesLT'):
    table_name.append(i.name.split('/')[0])

In [0]:
table_name

['Address',
 'Customer',
 'CustomerAddress',
 'Product',
 'ProductCategory',
 'ProductDescription',
 'ProductModel',
 'ProductModelProductDescription',
 'SalesOrderDetail',
 'SalesOrderHeader']

In [0]:
for i in table_name:
    path = f'/mnt/bronze/SalesLT/{i}/{i}.parquet'
    
    if dbutils.fs.ls(path):
        df = spark.read.format('parquet').load(path)
        columns = df.columns

        for col in columns:
            if "Date" in col or "date" in col:
                df = df.withColumn(
                    col, 
                    date_format(
                        from_utc_timestamp(df[col].cast(TimestampType()), "UTC"), 
                        "yyyy-MM-dd"
                    )
                )
        
        output_path = f'/mnt/silver/SalesLT/{i}/'
        df.write.format('delta').mode("overwrite").save(output_path)